In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
batch_size=64          #Batch size for training
epochs=100             #No. of epochs to train for
latent_dim=256         #Latent dimensionality of the encoding space.
num_samples=10000      #No. of samples to train on.
data_path='/content/drive/MyDrive/Data/fra.txt' #Path of the data file

In [6]:
#Vectorize the data
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path, 'r', encoding='utf-8') as f:
  lines=f.read().split('\n')
for line in lines[:min(num_samples,len(lines)-1)]:
  input_text, target_text,_=line.split('\t')
  input_texts.append(input_text)
  target_text='\t'+target_text+'\n'
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [7]:
type(input_text)

str

In [8]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [9]:
print('No. of samples:',len(input_texts))
print('No. of unique input tokens',num_encoder_tokens)
print('No. of unique output tokens',num_decoder_tokens)
print('No. of sequence length for inputs',max_encoder_seq_length)
print('No. of sequence length for outputs',max_decoder_seq_length)

No. of samples: 10000
No. of unique input tokens 71
No. of unique output tokens 92
No. of sequence length for inputs 15
No. of sequence length for outputs 59


In [10]:
input_token_index=dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index=dict([(char,i) for i,char in enumerate(target_characters)])

In [11]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

In [12]:
encoder_input_data=np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),
    dtype='float32'
)
decoder_input_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),
    dtype='float32'
)
decoder_target_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),
    dtype='float32'
)


In [13]:
encoder_input_data.shape[0]

10000

In [14]:
len(encoder_input_data)

10000

In [15]:
input_token_index['D']

23

In [16]:
#One Hot representation:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t,char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]]=1
  encoder_input_data[i,t+1:,input_token_index[' ']]=1
  for t,char in enumerate(target_text):
    decoder_input_data[i,t,target_token_index[char]]=1
    if t>0:
      decoder_target_data[i,t-1,target_token_index[char]]=1
  decoder_target_data[i,t:,target_token_index[' ']]=1
  decoder_input_data[i,t+1:,target_token_index[' ']]=1

In [17]:
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]
#Discarding the encoder outputs and keeping the states
encoder_states=[state_h,state_c]  

In [18]:
decoder_inputs= Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(latent_dim,return_sequences=True, return_state=True)
decoder_outputs, _, _=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense= Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [19]:
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 10s 17ms/step - loss: 1.1334 - val_loss: 1.0540
Epoch 2/100
125/125 [==============================] - 1s 11ms/step - loss: 0.8134 - val_loss: 0.8363
Epoch 3/100
125/125 [==============================] - 1s 11ms/step - loss: 0.6520 - val_loss: 0.7021
Epoch 4/100
125/125 [==============================] - 1s 11ms/step - loss: 0.5707 - val_loss: 0.6466
Epoch 5/100
125/125 [==============================] - 1s 11ms/step - loss: 0.5261 - val_loss: 0.5872
Epoch 6/100
125/125 [==============================] - 1s 11ms/step - loss: 0.4907 - val_loss: 0.5641
Epoch 7/100
125/125 [==============================] - 1s 11ms/step - loss: 0.4644 - val_loss: 0.5365
Epoch 8/100
125/125 [==============================] - 1s 11ms/step - loss: 0.4418 - val_loss: 0.5262
Epoch 9/100
125/125 [==============================] - 1s 11ms/step - loss: 0.4217 - val_loss: 0.5051
Epoch 10/100
125/125 [==============================] - 1s 12ms/step - loss: 0.40

In [20]:
# https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [21]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [26]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [27]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [38]:
for seq_index in range(10):

    input_seq = encoder_input_data[seq_index: seq_index + 1]
    #print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)




[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]
-
Input sentence: Go.
Decoded sentence: Bouge !

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]
-
Input sentence: Go.
Decoded sentence: Bouge !

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]
-
Input sentence: Go.
Decoded sentence: Bouge !

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]
-
Input sentence: Hi.
Decoded sentence: Salut !

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. 

In [29]:
a=np.zeros((1, 1, num_decoder_tokens))

In [30]:
a

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [40]:
#reverse_input_char_index
target_seq = np.zeros((1, 1, num_decoder_tokens))
target_seq[0, 0, target_token_index['\t']]=1

In [37]:
encoder_input_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [41]:
seq_index=0
input_seq = encoder_input_data[seq_index: seq_index + 1]

In [46]:
states_value = encoder_model.predict(input_seq)

In [47]:
 output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

In [48]:
output_tokens

array([[[3.65059591e-08, 9.05757418e-07, 2.87442708e-05, 4.49204678e-03,
         2.37309106e-09, 5.66014478e-07, 4.81329789e-06, 8.78851409e-08,
         1.85662099e-08, 5.34818264e-06, 5.26552103e-05, 9.64018400e-05,
         5.85604596e-07, 7.43770784e-07, 7.98754698e-08, 2.14724150e-08,
         7.29134229e-08, 1.05579304e-07, 2.94562142e-07, 4.43440172e-07,
         1.20263794e-05, 4.41969931e-02, 3.82300407e-01, 1.63324680e-02,
         6.36676252e-02, 4.80065246e-05, 1.25233945e-03, 1.06873158e-05,
         3.58719626e-05, 1.07947346e-02, 1.53266089e-02, 5.75962638e-08,
         2.82803420e-02, 1.17147222e-01, 1.26049446e-03, 2.07663011e-02,
         9.27939359e-03, 9.22097126e-04, 1.99514721e-03, 2.28504203e-02,
         7.00703869e-03, 1.20679233e-05, 1.59989566e-01, 4.77190006e-05,
         1.64859404e-03, 1.20376811e-07, 1.63205659e-05, 2.98490922e-04,
         1.09393443e-06, 2.63800939e-05, 2.36047617e-05, 1.01321871e-06,
         2.33003192e-07, 4.91944593e-06, 1.58715136

In [49]:
output_tokens[0, -1, :]

array([3.65059591e-08, 9.05757418e-07, 2.87442708e-05, 4.49204678e-03,
       2.37309106e-09, 5.66014478e-07, 4.81329789e-06, 8.78851409e-08,
       1.85662099e-08, 5.34818264e-06, 5.26552103e-05, 9.64018400e-05,
       5.85604596e-07, 7.43770784e-07, 7.98754698e-08, 2.14724150e-08,
       7.29134229e-08, 1.05579304e-07, 2.94562142e-07, 4.43440172e-07,
       1.20263794e-05, 4.41969931e-02, 3.82300407e-01, 1.63324680e-02,
       6.36676252e-02, 4.80065246e-05, 1.25233945e-03, 1.06873158e-05,
       3.58719626e-05, 1.07947346e-02, 1.53266089e-02, 5.75962638e-08,
       2.82803420e-02, 1.17147222e-01, 1.26049446e-03, 2.07663011e-02,
       9.27939359e-03, 9.22097126e-04, 1.99514721e-03, 2.28504203e-02,
       7.00703869e-03, 1.20679233e-05, 1.59989566e-01, 4.77190006e-05,
       1.64859404e-03, 1.20376811e-07, 1.63205659e-05, 2.98490922e-04,
       1.09393443e-06, 2.63800939e-05, 2.36047617e-05, 1.01321871e-06,
       2.33003192e-07, 4.91944593e-06, 1.58715136e-06, 1.32138623e-04,
      